In [7]:
import pandas as pd
from fuzzywuzzy import process,fuzz
import os
import os

def es_accesible(path):
    return os.path.isfile(path) and os.access(path, os.R_OK)

# Cargar CSV
df = pd.read_csv("spotify_songs_download.csv")

# Ruta de audios
audios_path = r"C:\Users\jefersson\Desktop\ANACONDA\audios"

# Lista y diccionario de archivos mp3
mp3_names = [
    os.path.splitext(f)[0].strip().lower()
    for f in os.listdir(audios_path)
    if f.endswith('.mp3')
]
mp3_dict = {
    name: os.path.abspath(os.path.join(audios_path, name + ".mp3"))
    for name in mp3_names
}

# Generar claves del CSV
df['clave'] = (df['track_artist'].str.strip() + " - " + df['track_name'].str.strip()).str.lower()

errores = []

for i, row in df.iterrows():
    clave = row['clave']
    match = process.extractOne(clave, mp3_names, scorer=fuzz.token_set_ratio)
    
    if match and match[1] >= 90 :
        matched_name = match[0]
        full_path = mp3_dict[matched_name]
        print(f"🕓 Anterior path: {row['path_download']}")
        print("es accesible path anterior:",es_accesible(row['path_download']))
        
        df.at[i, 'path_download'] = full_path
        print(f"✅ Nuevo path asignado: {full_path}")
        print(f"🔄 Match fuzzy: '{clave}' → '{matched_name}'\n")
       
    else:
        errores.append((i, clave))
        print(f"❌ Sin match: {clave}")

# Guardar el nuevo CSV
df.to_csv("spotify_songs_download_FIXED.csv", index=False)
print(f"\nProceso terminado. Sin match: {len(errores)} canciones.")


🕓 Anterior path: C:\Users\jefersson\Desktop\ANACONDA\audios\Ann Marie, YK Osiris - Secret (feat. YK Osiris).mp3
es accesible path anterior: True
✅ Nuevo path asignado: C:\Users\jefersson\Desktop\ANACONDA\audios\ann marie, yk osiris - secret (feat. yk osiris).mp3
🔄 Match fuzzy: 'ann marie - secret (feat. yk osiris)' → 'ann marie, yk osiris - secret (feat. yk osiris)'

🕓 Anterior path: C:\Users\jefersson\Desktop\ANACONDA\audios\Scorpions - Wind Of Change.mp3
es accesible path anterior: True
✅ Nuevo path asignado: C:\Users\jefersson\Desktop\ANACONDA\audios\scorpions - wind of change.mp3
🔄 Match fuzzy: 'scorpions - wind of change' → 'scorpions - wind of change'

🕓 Anterior path: C:\Users\jefersson\Desktop\ANACONDA\audios\Juan Pablo Vega, Elkin Robinson - Fishaman (feat. Elkin Robinson).mp3
es accesible path anterior: True
✅ Nuevo path asignado: C:\Users\jefersson\Desktop\ANACONDA\audios\juan pablo vega, elkin robinson - fishaman (feat. elkin robinson).mp3
🔄 Match fuzzy: 'juan pablo vega - 

In [17]:
# Obtener solo los índices de errores
indices_error = [i for i, _ in errores]
pd.set_option('display.max_colwidth', None)
# Mostrar las filas correspondientes
df.loc[indices_error, ['track_artist', 'track_name', 'path_download']]

error_persistente=[]
for i in indices_error:
    clave = df.at[i, 'clave']
    path_anterior = df.at[i, 'path_download']
    
    match = process.extractOne(clave, mp3_names, scorer=fuzz.token_set_ratio)
    
    if match and match[1] >= 82:
        matched_name = match[0]
        full_path = mp3_dict[matched_name]

        print(f"🔁 REASIGNADO (score={match[1]}%)")
        print(f"   🎵 Clave        : {clave}")
        print(f"   📂 Path anterior: {path_anterior}")
        print(f"   📥 Nuevo path   : {full_path}\n")

        # Actualizar
        df.at[i, 'path_download'] = full_path
    else:
        print(f"❌ SIN MATCH (score={match[1] if match else 'N/A'}): {clave}->{match[0]}\n")
        error_persistente.append(i)  # Aquí se guarda el índice


❌ SIN MATCH (score=73): slash - sweet child o' mine - feat. myles kennedy and the conspirators; live at spotify, london->guns n' roses - sweet child o' mine

❌ SIN MATCH (score=71): aj mitchell - all my friends - eden prince remix->prince - 1999

❌ SIN MATCH (score=80): pitbull - el taxi->iamchino, pitbull, yandel, el chacal - ay mi dios (feat. pitbull, yandel & chacal)

🔁 REASIGNADO (score=82%)
   🎵 Clave        : pitbull - timber (feat. ke$ha)
   📂 Path anterior: C:\Users\jefersson\Desktop\ANACONDA\audios\pitbull, kesha - timber.mp3
   📥 Nuevo path   : C:\Users\jefersson\Desktop\ANACONDA\audios\pitbull, kesha - timber.mp3

🔁 REASIGNADO (score=83%)
   🎵 Clave        : high school musical cast - the boys are back - original version
   📂 Path anterior: C:\Users\jefersson\Desktop\ANACONDA\audios\high school musical cast, corbin bleu, zac efron, disney - the boys are back.mp3
   📥 Nuevo path   : C:\Users\jefersson\Desktop\ANACONDA\audios\high school musical cast, corbin bleu, zac efron, d

In [32]:
print(len(error_persistente))

#duplicados = df[df.duplicated(subset='path_download', keep=False)]
#duplicados[['track_name', 'path_download']]
#df.to_csv("spotify_songs_download_FIXED.csv", index=False)
#print(f"\nProceso terminado. Sin match: {len(errores)} canciones.")

from fuzzywuzzy import fuzz
import os

# Subset de duplicados
duplicados = df[df.duplicated(subset='path_download', keep=False)]

# Lista para guardar los paths dudosos
paths_dudosos = []

for i, row in duplicados.iterrows():
    artista = row['track_artist'].strip().lower()
    titulo = row['track_name'].strip().lower()
    esperado = f"{artista} - {titulo}"

    # Obtener el nombre de archivo desde path_download
    path = row['path_download']
    if pd.isna(path):
        continue
    nombre_archivo = os.path.splitext(os.path.basename(path))[0].lower()

    score = fuzz.token_set_ratio(esperado, nombre_archivo)
    
    if score < 90:
        paths_dudosos.append((i, esperado, nombre_archivo, score))

# Mostrar resultados
for idx, esperado, archivo, score in paths_dudosos:
    print(f"⚠️ [{idx}] Score={score} → esperado: '{esperado}' -----> '{archivo}'")



41
⚠️ [334] Score=17 → esperado: 'connor bvrns - anthem' -----> 'chillwagon - flaga'
⚠️ [1507] Score=83 → esperado: 'dram - cash machine' -----> 'hard-fi - cash machine'
⚠️ [2232] Score=82 → esperado: 'pitbull - el taxi - radio edit' -----> 'jennifer lopez, pitbull - on the floor - radio edit'
⚠️ [2745] Score=86 → esperado: 'jamie berry - delight - original mix' -----> 'jamie berry, rosie harte - peeping tom - original mix'
⚠️ [2837] Score=84 → esperado: 'hardwell - arcadia - original mix' -----> 'hardwell, makj - countdown - original mix'


In [11]:
import pandas as pd
# Cargar CSV
df = pd.read_csv("spotify_songs_download.csv")

df['path_download'] = df['path_download'].str.replace(
    r'C:\\Users\\jefersson\\Desktop\\ANACONDA', '.', regex=True
).str.replace('\\', '/')

df['path_download_wav'] = df['path_download_wav'].str.replace(
    r'C:\\Users\\jefersson\\Desktop\\ANACONDA', '.', regex=True
).str.replace('\\', '/')


df.to_csv('spotify_songs_download.csv')


In [10]:
df[['path_download', 'path_download_wav']]


,path_download,path_download_wav
0,"./audios/ann marie, yk osiris - secret (feat. ...","./audios_wav/ann marie, yk osiris - secret (fe..."
1,./audios/scorpions - wind of change.mp3,./audios_wav/scorpions - wind of change.wav
2,"./audios/juan pablo vega, elkin robinson - fis...","./audios_wav/juan pablo vega, elkin robinson -..."
3,./audios/the bolshoi - sunday morning.mp3,./audios_wav/the bolshoi - sunday morning.wav
4,./audios/otherwise - die for you.mp3,./audios_wav/otherwise - die for you.wav
...,...,...
2908,"./audios/matthew koma, vicetone - one night - ...","./audios_wav/matthew koma, vicetone - one nigh..."
2909,"./audios/cosmic gate, pelari - so get up - pel...","./audios_wav/cosmic gate, pelari - so get up -..."
2910,./audios/jessie j - masterpiece.mp3,./audios_wav/jessie j - masterpiece.wav
2911,"./audios/regard, jonas blue - ride it - jonas ...","./audios_wav/regard, jonas blue - ride it - jo..."


In [1]:
import pandas as pd
import numpy as np

# Cargar CSV
df = pd.read_csv("spotify_songs_download.csv")

# Crear una columna 'id' con valores 1, 2, ..., n
df['id'] = np.arange(1, len(df) + 1)


In [2]:
df['id']

0          1
1          2
2          3
3          4
4          5
        ... 
2908    2909
2909    2910
2910    2911
2911    2912
2912    2913
Name: id, Length: 2913, dtype: int64

In [3]:
df.to_csv("spotify_songs_download.csv")

In [3]:
import pandas as pd

csv_path = "./spotify_songs_download_FINAL.csv"

df = pd.read_csv(csv_path)

if 'clave' in df.columns:
    df = df.drop(columns=['clave'])
    df.to_csv(csv_path, index=False)
    print("✅ Columna 'clave' eliminada.")
else:
    print("✅ No existe la columna 'clave.")


✅ No existe la columna 'clave.


In [4]:
#UNIR CSV 
csv_path1 = "./spotify_songs_download_FINAL.csv"
csv_path2 = "./spotify_songs_download.csv"

df1 = pd.read_csv(csv_path1)
df2=  pd.read_csv(csv_path2)


# join by track_id
merge=pd.merge(df1,df2,on="track_id",how="inner")
print(len(df))
df_merge=merge[["track_id", "track_album_id", "id", "track_artist_x", "track_name_x", "playlist_genre", "playlist_subgenre", "path_download_wav", "lyrics"]]

df_merge.rename(columns={
    "track_artist_x":"track_artist",
    "track_name_x":"track_name",
},inplace=True )


2913


C:\Users\jefersson\AppData\Local\Temp\ipykernel_26132\1646909124.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge.rename(columns={


In [5]:
df_merge.to_csv("recomendaciones_final.csv",index=False)